In [1]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [2]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [2]:
session = boto3.Session(profile_name='master')

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [3]:
import awswrangler as wr


In [4]:
s3 = session.client('s3')

In [70]:
def get_json_from_s3(bucket, key, session):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [39]:
res_empty = get_json_from_s3("kosmobiker-masterproject", 'data/raw/ape_in_records_historical_prices.json', session)
res_bit = get_json_from_s3("kosmobiker-masterproject", 'data/raw/bitcoin_historical_prices.json', session)
res_amar = get_json_from_s3("kosmobiker-masterproject", 'data/raw/amateras_historical_prices.json', session)
res_aaa = get_json_from_s3("kosmobiker-masterproject", 'data/raw/american-shiba_historical_prices.json', session)

In [57]:
len(res_empty['prices'])

1

In [61]:
def make_df(json_data):
    return pd.DataFrame({
                "timestamp" : [i[0] for i in json_data['prices']],
                "prices" : [i[1] for i in json_data['prices']],
                "market_caps" : [i[1] for i in json_data['market_caps']],
                "total_volumes" : [i[1] for i in json_data['total_volumes']]
    })
    


In [36]:
import jsonschema
def validate_json(data, schema):
    try:
        validate(instance=data, schema=schema)
        if len(data['prices']) < 30:
            raise ValueError
        return True
    except Exception:
        return False



In [104]:
s3 = session.client('s3')

def ListFilesV1(client, bucket, prefix=''):
    """List files in specific S3 URL"""
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket,
                                    Prefix=prefix,
                                    PaginationConfig={'MaxItems': 100})
    for page in page_iterator:
        for content in page.get('Contents'):
            yield content.get('Key')
gen = ListFilesV1(s3, 'kosmobiker-masterproject', prefix="data/raw")

In [105]:
for _ in gen:
    key = next(gen)
    print(key)

data/raw/0-5x-long-altcoin-index-token_historical_prices.json
data/raw/0-5x-long-bitcoin-cash-token_historical_prices.json
data/raw/0-5x-long-cardano-token_historical_prices.json
data/raw/0-5x-long-cosmos-token_historical_prices.json
data/raw/0-5x-long-eos-token_historical_prices.json
data/raw/0-5x-long-ethereum-token_historical_prices.json
data/raw/0-5x-long-matic-token_historical_prices.json
data/raw/0-5x-long-swipe-token_historical_prices.json
data/raw/0-5x-long-xrp-token_historical_prices.json
data/raw/0cash_historical_prices.json
data/raw/0x-wormhole_historical_prices.json
data/raw/0xcert_historical_prices.json
data/raw/0xmonero_historical_prices.json
data/raw/10024-10028-appoline_historical_prices.json
data/raw/100man_historical_prices.json
data/raw/amasa_historical_prices.json
data/raw/amateras_historical_prices.json
data/raw/amazon-tokenized-stock-defichain_historical_prices.json
data/raw/amber_historical_prices.json
data/raw/amc-fight-night_historical_prices.json
data/raw/amer